# Elementos escenciales

Alberga la base del sistema y la conexión.

## Librerías

Importamos las librerías necesarias para el funcionamiento del código.

In [15]:
import oracledb
import os 
from pathlib import Path
from datetime import datetime
from typing import Optional

## Conexión reutilizable

Se crea una función para conectarse a la base de datos local.

In [16]:
def get_connection():
    return oracledb.connect(
        user="SYSTEM",
        password="96443804",
        dsn="localhost:1521/xe"
    ) 

# Creación de la base de datos

Código necesario para crear las tablas que representan las entidades de nuestro modelo.

## Esquema base de datos

Se define una función para crear todas las tablas a partir de una serie de consultas.

In [17]:
def create_schema(query):
    try:         
        with get_connection() as conn:             
            with conn.cursor() as cur:                 
                cur.execute(query)                 
                print(f'✓ Tabla creada exitosamente')
            conn.commit()     
    except oracledb.DatabaseError as e:        
        err = e
        if 'ORA-00955' in str(err):  # Tabla ya existe
            pass  # No imprimir nada si la tabla ya existe
        else:
            print(f"Error al crear tabla: {err}")

## Tablas de la base de datos

Se crea una función que alberga las consultas necesarias para la creación de las tablas y que llama la función para crear las tablas.

In [18]:
def create_all_tables():
    tables = [
        (
            "CREATE TABLE "
            "PERSONAS ("
            "RUT VARCHAR(10) PRIMARY KEY,"
            "NOMBRES VARCHAR(64),"
            "APELLIDOS VARCHAR(64))"
        ),
        (
            "CREATE TABLE "
            "ASIGNATURAS ("
            "ID INTEGER PRIMARY KEY,"
            "NOMBRE VARCHAR(32))"
        ),
        (
            "CREATE TABLE "
            "PROFESORES ("
            "ID INTEGER PRIMARY KEY,"
            "PROFESOR VARCHAR(10),"
            "ASIGNATURA INTEGER,"
            "FOREIGN KEY (PROFESOR) REFERENCES PERSONAS(RUT),"
            "FOREIGN KEY (ASIGNATURA) REFERENCES ASIGNATURAS(ID))"
        ),
        (
            "CREATE TABLE "
            "CURSOS ("
            "ID VARCHAR(10) PRIMARY KEY,"
            "ALUMNO VARCHAR(10),"
            "FOREIGN KEY (ALUMNO) REFERENCES PERSONAS(RUT))"
        ),
        (
            "CREATE TABLE "
            "HORARIOS ("
            "ID INTEGER PRIMARY KEY,"
            "PROFESOR INTEGER,"
            "CURSO VARCHAR(10),"
            "FOREIGN KEY (PROFESOR) REFERENCES PROFESORES(ID),"
            "FOREIGN KEY (CURSO) REFERENCES CURSOS(ID))"
        ),
        (
            "CREATE TABLE "
            "EVALUACIONES ("
            "ID INTEGER PRIMARY KEY,"
            "NOTA INTEGER,"
            "ASIGNATURA INTEGER,"
            "ALUMNO VARCHAR(10),"
            "PROFESOR INTEGER,"
            "FOREIGN KEY (ASIGNATURA) REFERENCES ASIGNATURAS(ID),"
            "FOREIGN KEY (ALUMNO) REFERENCES PERSONAS(RUT),"
            "FOREIGN KEY (PROFESOR) REFERENCES PROFESORES(ID))"
        )
    ]
    for query in tables:
        create_schema(query)

## Función para limpiar tablas previamente creadas

Para agilizar el proceso de testeo y demostración, se creó una función que limpiará la base de datos si es que esta estaba previamente poblada.

In [19]:
def drop_all_tables():
    try:
        # Use context managers so connections and cursors are closed automatically
        with get_connection() as conn:
            with conn.cursor() as cur:
                # Orden de eliminación (respetando foreign keys)
                tablas = ['EVALUACIONES', 'HORARIOS', 'CURSOS', 'PROFESORES', 'ASIGNATURAS', 'PERSONAS']

                for tabla in tablas:
                    try:
                        cur.execute(f'DROP TABLE {tabla}')
                        conn.commit()
                        print(f'✓ Tabla {tabla} eliminada')
                    except oracledb.DatabaseError as e:
                        if 'ORA-00942' in str(e):  # Tabla no existe
                            print(f'- Tabla {tabla} no existe (ignorado)')
                        else:
                            print(f'✗ Error al eliminar {tabla}: {e}')

        print("\n✓ Limpieza completada exitosamente")

    except oracledb.DatabaseError as e:
        print(f"Error de conexión: {e}")

# Funciones para insertar datos

Debido a que la estructura de cada tabla varía, es necesario crear funciones específicas para insertar datos en la entidad correspondiente.

## Tabla Personas

In [20]:
def create_persona(rut=None, nombres=None, apellidos=None):     
    sql = (        
    "INSERT INTO personas (rut, nombres, apellidos) "         
    "VALUES (:rut, :nombres, :apellidos)"     
    )     
    try:
        with get_connection() as conn:         
            with conn.cursor() as cur:             
                cur.execute(sql, {                 
                "rut": rut,                 
                "nombres": nombres,                 
                "apellidos": apellidos             
                })             
            conn.commit()            
        print(f"✓ Persona con RUT={rut} creada.")
    except oracledb.DatabaseError as e:
        print(f"Error al crear persona: {e}") 

## Tabla Asignaturas

In [21]:
def create_asignatura(id=None, nombre=None):     
    sql = (        
    "INSERT INTO ASIGNATURAS (id, nombre)"         
    "VALUES (:id, :nombre)"     
    )    
    with get_connection() as conn:         
        with conn.cursor() as cur:             
            cur.execute(sql, {                 
            "id": id,                 
            "nombre": nombre          
            })             
        conn.commit()            
    print(f"Departamento {nombre} creado.") 

## Tabla Profesores

In [22]:
def create_profesor(ID=None, PROFESOR=None, ASIGNATURA=None):     
    sql = (        
    "INSERT INTO PROFESORES (ID, PROFESOR, ASIGNATURA)"         
    "VALUES (:ID, :PROFESOR, :ASIGNATURA)"     
    )    
    with get_connection() as conn:         
        with conn.cursor() as cur:             
            cur.execute(sql, {                 
            "ID": ID,
            "PROFESOR": PROFESOR,                 
            "ASIGNATURA": ASIGNATURA        
            })             
        conn.commit()            
    print(f"Profesor {PROFESOR} registrado.") 

## Tabla Cursos

In [23]:
def create_curso(ID=None, ALUMNO=None):     
    sql = (        
    "INSERT INTO CURSOS (ID, ALUMNO)"         
    "VALUES (:ID, :ALUMNO)"     
    )    
    with get_connection() as conn:         
        with conn.cursor() as cur:             
            cur.execute(sql, {                 
            "ID": ID,                 
            "ALUMNO": ALUMNO        
            })             
        conn.commit()            
    print(f"Alumno {ALUMNO} registrado en curso {ID}.")     


## Tabla Horarios

In [24]:
def create_horario(ID=None, PROFESOR=None, CURSO=None):     
    sql = (        
    "INSERT INTO HORARIOS (ID, PROFESOR, CURSO)"         
    "VALUES (:ID, :PROFESOR, :CURSO)"     
    )    
    with get_connection() as conn:         
        with conn.cursor() as cur:             
            cur.execute(sql, {                 
            "ID": ID,                 
            "PROFESOR": PROFESOR,
            "CURSO": CURSO    
            })             
        conn.commit()            
    print(f"Profesor {PROFESOR} dictará curso {CURSO}.")    


## Tabla Evaluaciones

In [25]:
def create_evaluacion(ID=None, NOTA=None, ASIGNATURA=None, ALUMNO=None, PROFESOR=None):     
    sql = (        
    "INSERT INTO EVALUACIONES (ID, NOTA, ASIGNATURA, ALUMNO, PROFESOR)"         
    "VALUES (:ID, :NOTA, :ASIGNATURA, :ALUMNO, :PROFESOR)"     
    )    
    with get_connection() as conn:         
        with conn.cursor() as cur:             
            cur.execute(sql, {                 
            "ID": ID,
            "NOTA": NOTA,
            "ASIGNATURA": ASIGNATURA,
            "ALUMNO": ALUMNO,                 
            "PROFESOR": PROFESOR 
            })             
        conn.commit()            
    print(f"Alumno {ALUMNO} recibió la nota {NOTA}.") 


## Poblamiento de tablas

### Poblar tabla Personas

In [26]:
create_persona("12345678-9", "Juan", "Pérez")
create_persona("23456789-0", "María", "González")
create_persona("34567890-1", "Carlos", "López")
create_persona("45678901-K", "Ana", "Martínez")
create_persona("56789012-3", "Pedro", "Sánchez")
create_persona("67890123-4", "Laura", "Rodríguez")

✓ Persona con RUT=12345678-9 creada.
✓ Persona con RUT=23456789-0 creada.
✓ Persona con RUT=34567890-1 creada.
✓ Persona con RUT=45678901-K creada.
✓ Persona con RUT=56789012-3 creada.
✓ Persona con RUT=67890123-4 creada.


### Poblar tabla Asignaturas

In [27]:
drop_all_tables()
create_all_tables()
create_asignatura(1, "Matemáticas")
create_asignatura(2, "Lenguaje")
create_asignatura(3, "Historia")
create_asignatura(4, "Ciencias")
create_asignatura(5, "Educación Física")

✓ Tabla EVALUACIONES eliminada
✓ Tabla HORARIOS eliminada
✓ Tabla CURSOS eliminada
✓ Tabla PROFESORES eliminada
✓ Tabla ASIGNATURAS eliminada
✓ Tabla PERSONAS eliminada

✓ Limpieza completada exitosamente
✓ Tabla creada exitosamente
✓ Tabla creada exitosamente
✓ Tabla creada exitosamente
✓ Tabla creada exitosamente
✓ Tabla creada exitosamente
✓ Tabla creada exitosamente
Departamento Matemáticas creado.
Departamento Lenguaje creado.
✓ Tabla creada exitosamente
✓ Tabla creada exitosamente
✓ Tabla creada exitosamente
Departamento Matemáticas creado.
Departamento Lenguaje creado.
Departamento Historia creado.
Departamento Ciencias creado.
Departamento Educación Física creado.
Departamento Historia creado.
Departamento Ciencias creado.
Departamento Educación Física creado.


### Poblar tabla Profesores

In [28]:
create_profesor(1, "12345678-9", 1)
create_profesor(2, "23456789-0", 2)
create_profesor(3, "34567890-1", 3)
create_profesor(4, "45678901-K", 4)
create_profesor(5, "56789012-3", 5)

IntegrityError: ORA-02291: restricción de integridad (SYSTEM.SYS_C008488) violada - clave principal no encontrada
Help: https://docs.oracle.com/error-help/db/ora-02291/

### Poblar tabla Cursos

In [ ]:
create_curso("3-B", "67890123-4")
create_curso("6-A", "56789012-3")
create_curso("5-B", "45678901-K")
create_curso("7-A", "34567890-1")
create_curso("IV-A", "23456789-0")

### Poblar tabla Horarios

In [ ]:
create_horario(1, 1, "3-B")
create_horario(2, 2, "6-A")
create_horario(3, 3, "5-B")
create_horario(4, 4, "7-A")
create_horario(5, 5, "IV-A")

### Poblar tabla Evaluaciones

In [ ]:
create_evaluacion(1, 95, 1, "67890123-4", 1)
create_evaluacion(2, 87, 2, "56789012-3", 2)
create_evaluacion(3, 92, 3, "45678901-K", 3)
create_evaluacion(4, 78, 4, "34567890-1", 4)
create_evaluacion(5, 88, 5, "23456789-0", 5)

# Funciones para leer datos

Se definen funciones para obtener todos los datos de una tabla determinada o para buscar un elemento de una tabla determinada.

## Leer todos los datos de una tabla

In [ ]:
def read_table(tabla):
    sql = (
        f"SELECT * FROM {tabla}"
    )
    try:
        with get_connection() as conn:
            with conn.cursor() as cur:
                resultados = cur.execute(sql)
                print(f"Consulta a la tabla {tabla}")
                for fila in resultados:
                    print(fila)
    except oracledb.DatabaseError as e:
        err = e
        print(f"Error al leer datos: {err}")

## Buscar por identificador

In [ ]:
def read_table_by_id(tabla, columna_id, valor_id):
    sql = (
        f"SELECT * FROM {tabla} WHERE {columna_id} = :valor"
    )
    parametros = {"valor": valor_id}
    try:
        with get_connection() as conn:
            with conn.cursor() as cur:
                resultados = cur.execute(sql, parametros)
                print(f"✓ Consulta a la tabla {tabla} donde {columna_id} = {valor_id}")
                encontrados = 0
                for fila in resultados:
                    print(f"  {fila}")
                    encontrados += 1
                if encontrados == 0:
                    print(f"  No se encontraron registros")
    except oracledb.DatabaseError as e:
        print(f"Error al leer datos: {e}")

# Funciones para actualizar datos

## Actualizar Persona

In [ ]:
def update_persona(
    RUT,
    NOMBRES: Optional[str] = None,
    APELLIDOS: Optional[str] = None
):
    modificaciones = []
    parametros = {"RUT": RUT}

    if RUT is not None:
        modificaciones.append("RUT =: RUT")
        parametros["RUT"] = RUT
    if NOMBRES is not None:
        modificaciones.append("NOMBRES =: NOMBRES")
        parametros["NOMBRES"] = NOMBRES
    if APELLIDOS is not None:
        modificaciones.append("APELLIDOS =: APELLIDOS")
        parametros["APELLIDOS"] = APELLIDOS
    if not modificaciones:
        return print("No hay campos para actualizar.")

    sql = f"UPDATE PERSONAS SET {", ".join(modificaciones)} WHERE RUT =: RUT"

    with get_connection() as conn:
        with conn.cursor() as cur:
            cur.execute(sql, parametros)
        conn.commit()
        print(f"Persona con RUT={RUT} actualizada.")

## Actualizar Asignatura

In [ ]:
def update_asignatura(
    ID: int,
    NOMBRE: Optional[str] = None
):
    modificaciones = []
    parametros = {"ID": ID}

    if NOMBRE is not None:
        modificaciones.append("NOMBRE = :NOMBRE")
        parametros["NOMBRE"] = NOMBRE

    if not modificaciones:
        print("No hay campos para actualizar en Asignaturas.")
        return

    sql = f"UPDATE ASIGNATURAS SET {', '.join(modificaciones)} WHERE ID = :ID"
    with get_connection() as conn:
        with conn.cursor() as cur:
            cur.execute(sql, parametros)
        conn.commit()
    print(f"Asignatura ID={ID} actualizada.")

## Actualizar Profesor

In [ ]:
def update_profesor(
    ID: int,
    PROFESOR: Optional[str] = None,
    ASIGNATURA: Optional[int] = None
):
    modificaciones = []
    parametros = {"ID": ID}

    if PROFESOR is not None:
        modificaciones.append("PROFESOR = :PROFESOR")
        parametros["PROFESOR"] = PROFESOR
    if ASIGNATURA is not None:
        modificaciones.append("ASIGNATURA = :ASIGNATURA")
        parametros["ASIGNATURA"] = ASIGNATURA

    if not modificaciones:
        print("No hay campos para actualizar en Profesores.")
        return

    sql = f"UPDATE PROFESORES SET {', '.join(modificaciones)} WHERE ID = :ID"
    with get_connection() as conn:
        with conn.cursor() as cur:
            cur.execute(sql, parametros)
        conn.commit()
    print(f"Profesor ID={ID} actualizado.")

## Actualizar Curso

In [ ]:
def update_curso(
    ID: str,
    ALUMNO: Optional[str] = None
):
    modificaciones = []
    parametros = {"ID": ID}

    if ALUMNO is not None:
        modificaciones.append("ALUMNO = :ALUMNO")
        parametros["ALUMNO"] = ALUMNO

    if not modificaciones:
        print("No hay campos para actualizar en Cursos.")
        return

    sql = f"UPDATE CURSOS SET {', '.join(modificaciones)} WHERE ID = :ID"
    with get_connection() as conn:
        with conn.cursor() as cur:
            cur.execute(sql, parametros)
        conn.commit()
    print(f"Curso ID={ID} actualizado.")

## Actualizar Horario

In [ ]:
def update_horario(
    ID: int,
    PROFESOR: Optional[int] = None,
    CURSO: Optional[str] = None
):
    modificaciones = []
    parametros = {"ID": ID}

    if PROFESOR is not None:
        modificaciones.append("PROFESOR = :PROFESOR")
        parametros["PROFESOR"] = PROFESOR
    if CURSO is not None:
        modificaciones.append("CURSO = :CURSO")
        parametros["CURSO"] = CURSO

    if not modificaciones:
        print("No hay campos para actualizar en Horarios.")
        return

    sql = f"UPDATE HORARIOS SET {', '.join(modificaciones)} WHERE ID = :ID"
    with get_connection() as conn:
        with conn.cursor() as cur:
            cur.execute(sql, parametros)
        conn.commit()
    print(f"Horario ID={ID} actualizado.")

## Actualizar Evaluación

In [ ]:
def update_evaluacion(
    ID: int,
    NOTA: Optional[int] = None,
    ASIGNATURA: Optional[int] = None,
    ALUMNO: Optional[str] = None,
    PROFESOR: Optional[int] = None
):
    modificaciones = []
    parametros = {"ID": ID}

    if NOTA is not None:
        modificaciones.append("NOTA = :NOTA")
        parametros["NOTA"] = NOTA
    if ASIGNATURA is not None:
        modificaciones.append("ASIGNATURA = :ASIGNATURA")
        parametros["ASIGNATURA"] = ASIGNATURA
    if ALUMNO is not None:
        modificaciones.append("ALUMNO = :ALUMNO")
        parametros["ALUMNO"] = ALUMNO
    if PROFESOR is not None:
        modificaciones.append("PROFESOR = :PROFESOR")
        parametros["PROFESOR"] = PROFESOR

    if not modificaciones:
        print("No hay campos para actualizar en Evaluaciones.")
        return

    sql = f"UPDATE EVALUACIONES SET {', '.join(modificaciones)} WHERE ID = :ID"
    with get_connection() as conn:
        with conn.cursor() as cur:
            cur.execute(sql, parametros)
        conn.commit()
    print(f"Evaluación ID={ID} actualizada.")

# Funciones para eliminar datos

## Eliminar Persona

In [ ]:
def delete_persona(RUT):
    sql = (
        "DELETE FROM PERSONAS WHERE RUT = :RUT"
    )
    parametros = {"RUT" : RUT}
    try:
        with get_connection() as conn:
            with conn.cursor() as cur:
                cur.execute(sql, parametros)
            conn.commit()
            print(f"Persona con RUT {RUT} eliminada.")
    except oracledb.DatabaseError as e:
        err = e
        print(f"Error al eliminar dato: {err} \n {sql} \n {parametros}")

## Eliminar Asignatura

In [ ]:
def delete_asignatura(ID: int):
    sql = "DELETE FROM ASIGNATURAS WHERE ID = :ID"
    parametros = {"ID": ID}
    try:
        with get_connection() as conn:
            with conn.cursor() as cur:
                cur.execute(sql, parametros)
            conn.commit()
        print(f"Asignatura ID={ID} eliminada.")
    except oracledb.DatabaseError as e:
        print(f"Error al eliminar asignatura ID={ID}: {e}")

## Eliminar Profesor

In [ ]:
def delete_profesor(ID: int):
    sql = "DELETE FROM PROFESORES WHERE ID = :ID"
    parametros = {"ID": ID}
    try:
        with get_connection() as conn:
            with conn.cursor() as cur:
                cur.execute(sql, parametros)
            conn.commit()
        print(f"Profesor ID={ID} eliminado.")
    except oracledb.DatabaseError as e:
        print(f"Error al eliminar profesor ID={ID}: {e}")

## Eliminar Curso

In [ ]:
def delete_curso(ID: str):
    sql = "DELETE FROM CURSOS WHERE ID = :ID"
    parametros = {"ID": ID}
    try:
        with get_connection() as conn:
            with conn.cursor() as cur:
                cur.execute(sql, parametros)
            conn.commit()
        print(f"Curso ID={ID} eliminado.")
    except oracledb.DatabaseError as e:
        print(f"Error al eliminar curso ID={ID}: {e}")

## Eliminar Horario

In [ ]:
def delete_horario(ID: int):
    sql = "DELETE FROM HORARIOS WHERE ID = :ID"
    parametros = {"ID": ID}
    try:
        with get_connection() as conn:
            with conn.cursor() as cur:
                cur.execute(sql, parametros)
            conn.commit()
        print(f"Horario ID={ID} eliminado.")
    except oracledb.DatabaseError as e:
        print(f"Error al eliminar horario ID={ID}: {e}")

## Eliminar Evaluación

In [ ]:
def delete_evaluacion(ID: int):
    sql = "DELETE FROM EVALUACIONES WHERE ID = :ID"
    parametros = {"ID": ID}
    try:
        with get_connection() as conn:
            with conn.cursor() as cur:
                cur.execute(sql, parametros)
            conn.commit()
        print(f"Evaluación ID={ID} eliminada.")
    except oracledb.DatabaseError as e:
        print(f"Error al eliminar evaluación ID={ID}: {e}")

# Menú de Usuario

In [ ]:
def __main__():
    while True:
        print(
            """
            ==========================================
            |      Sistema de Gestión Académico      |
            ==========================================
            |    Bienvenido al sistema de gestión    |
            |          ¿Qué deseas hacer?:           |
            ==========================================
            | 1. Ingresar datos                      |
            | 2. Consultar datos                     |
            | 3. Actualizar datos                    |
            | 4. Eliminar datos                      |
            | 0. Salir                               |
            ==========================================
            """
        )
        opcion = input("Selecciona una opcion [1-4, 0 para salir]:")
        if opcion == "0":
            print("Saliendo del sistema...")
            break
        elif opcion == "1":
            print(
                """
                ==========================================
                |      Sistema de Gestión Académico      |
                ==========================================
                |   ¿Dónde deseas ingresar los datos?    |
                ==========================================
                | 1. Tabla Personas                      |
                | 2. Tabla Asignatura                    |
                | 3. Tabla Profesores                    |
                | 4. Tabla Cursos                        |
                | 5. Tabla Horarios                      |
                | 6. Tabla Evaluaciones                  |
                | 0. Salir                               |
                ==========================================
                """
            )
            opcion_ingresar = input("Selecciona una opcion [1-6, 0 para salir]:")
            if opcion_ingresar == "0":
                print("Regresando...")
            elif opcion_ingresar == "1":
                rut = input("Ingrese RUT: ")
                nombres = input("Ingrese Nombres: ")
                apellidos = input("Ingrese Apellidos: ")
                create_persona(rut, nombres, apellidos)
            elif opcion_ingresar == "2":
                id = int(input("Ingrese ID de Asignatura: "))
                nombre = input("Ingrese Nombre de Asignatura: ")
                create_asignatura(id, nombre)
            elif opcion_ingresar == "3":
                id = int(input("Ingrese ID de Profesor: "))
                profesor = input("Ingrese RUT del Profesor: ")
                asignatura = int(input("Ingrese ID de Asignatura: "))
                create_profesor(id, profesor, asignatura)
            elif opcion_ingresar == "4":
                id = input("Ingrese ID de Curso: ")
                alumno = input("Ingrese RUT del Alumno: ")
                create_curso(id, alumno)
            elif opcion_ingresar == "5":
                id = int(input("Ingrese ID de Horario: "))
                profesor = int(input("Ingrese ID de Profesor: "))
                curso = input("Ingrese ID de Curso: ")
                create_horario(id, profesor, curso)
            elif opcion_ingresar == "6":
                id = int(input("Ingrese ID de Evaluación: "))
                nota = int(input("Ingrese Nota: "))
                asignatura = int(input("Ingrese ID de Asignatura: "))
                alumno = input("Ingrese RUT del Alumno: ")
                profesor = int(input("Ingrese ID de Profesor: "))
                create_evaluacion(id, nota, asignatura, alumno, profesor)
            else:
                print("Opción inválida. Intente nuevamente.")

        elif opcion == "2":
            print(
                """
                ==========================================
                |      Sistema de Gestión Académico      |
                ==========================================
                |     ¿Cómo deseas consultar los datos?  |
                ==========================================
                | 1. Quiero consultar todos los datos    |
                |    de una tabla.                       |
                | 2. Quiero consultar un dato            |
                |    específico.                         |
                | 0. Salir                               |
                ==========================================
                """
            )
            opcion_consultar = input("Selecciona una opción [1-2, 0 para salir]:")
            if opcion_consultar == "0":
                print("Regresando...")
            elif opcion_consultar == "1":
                print(
                    """
                    ==========================================
                    |      Sistema de Gestión Académico      |
                    ==========================================
                    |      ¿Qué tabla deseas consultar?      |
                    ==========================================
                    | 1. Tabla Personas                      |
                    | 2. Tabla Asignatura                    |
                    | 3. Tabla Profesores                    |
                    | 4. Tabla Cursos                        |
                    | 5. Tabla Horarios                      |
                    | 6. Tabla Evaluaciones                  |
                    | 0. Salir                               |
                    ==========================================
                    """
                )
                opcion_tabla = input("Selecciona una opción [1-6, 0 para salir]")
                if opcion_tabla == "0":
                    print("Regresando...")
                if opcion_tabla == "1":
                    read_table("PERSONAS")
                elif opcion_tabla == "2":
                    read_table("ASIGNATURAS")
                elif opcion_tabla == "3":
                    read_table("PROFESORES")
                elif opcion_tabla == "4":
                    read_table("CURSOS")
                elif opcion_tabla == "5":
                    read_table("HORARIOS")
                elif opcion_tabla == "6":
                    read_table("EVALUACIONES")
                else:
                    print("Opción inválida. Intente nuevamente.")
            
            elif opcion_consultar == "2":
                tabla=upper(input("¿Qué tabla deseas consultar?: [Personas - Asignatura - Profesores - Cursos - Horarios - Evaluaciones]"))
                parametro=upper(input("¿Por cuál parámetro buscarás el dato?: [Id - RUT - Nombre - ETC]"))
                valor=upper(input("Ingresa el identificador: "))
                read_table_by_id(tabla, parametro, valor)
        
        elif opcion == "3":
            print(
                """
                ==========================================
                |      Sistema de Gestión Académico      |
                ==========================================
                |   ¿Dónde deseas actualizar los datos?  |
                ==========================================
                | 1. Tabla Personas                      |
                | 2. Tabla Asignatura                    |
                | 3. Tabla Profesores                    |
                | 4. Tabla Cursos                        |
                | 5. Tabla Horarios                      |
                | 6. Tabla Evaluaciones                  |
                | 0. Salir                               |
                ==========================================
                """
            )
            opcion_actualizar = input("Selecciona una opcion [1-6, 0 para salir]:")
            if opcion_actualizar == "0":
                print("Regresando...")
            elif opcion_actualizar == "1":
                rut = input("Ingrese RUT de la persona a actualizar: ")
                nombres = input("Ingrese nuevos Nombres (dejar en blanco para no cambiar): ")
                apellidos = input("Ingrese nuevos Apellidos (dejar en blanco para no cambiar): ")
                update_persona(
                    RUT=rut,
                    NOMBRES=nombres if nombres else None,
                    APELLIDOS=apellidos if apellidos else None
                )
            elif opcion_actualizar == "2":
                id = int(input("Ingrese ID de la asignatura a actualizar: "))
                nombre = input("Ingrese nuevo Nombre (dejar en blanco para no cambiar): ")
                update_asignatura(
                    ID=id,
                    NOMBRE=nombre if nombre else None
                )
            elif opcion_actualizar == "3":
                id = int(input("Ingrese ID del profesor a actualizar: "))
                profesor = input("Ingrese nuevo RUT del Profesor (dejar en blanco para no cambiar): ")
                asignatura = input("Ingrese nuevo ID de Asignatura (dejar en blanco para no cambiar): ")
                update_profesor(
                    ID=id,
                    PROFESOR=profesor if profesor else None,
                    ASIGNATURA=int(asignatura) if asignatura else None
                )
            elif opcion_actualizar == "4":
                id = input("Ingrese ID del curso a actualizar: ")
                alumno = input("Ingrese nuevo RUT del Alumno (dejar en blanco para no cambiar): ")
                update_curso(
                    ID=id,
                    ALUMNO=alumno if alumno else None
                )
            elif opcion_actualizar == "5":
                id = int(input("Ingrese ID del horario a actualizar: "))
                profesor = input("Ingrese nuevo ID del Profesor (dejar en blanco para no cambiar): ")
                curso = input("Ingrese nuevo ID del Curso (dejar en blanco para no cambiar): ")
                update_horario(
                    ID=id,
                    PROFESOR=int(profesor) if profesor else None,
                    CURSO=curso if curso else None
                )
            elif opcion_actualizar == "6":
                id = int(input("Ingrese ID de la evaluación a actualizar: "))
                nota = input("Ingrese nueva Nota (dejar en blanco para no cambiar): ")
                asignatura = input("Ingrese nuevo ID de Asignatura (dejar en blanco para no cambiar): ")
                alumno = input("Ingrese nuevo RUT del Alumno (dejar en blanco para no cambiar): ")
                profesor = input("Ingrese nuevo ID del Profesor (dejar en blanco para no cambiar): ")
                update_evaluacion(
                    ID=id,
                    NOTA=int(nota) if nota else None,
                    ASIGNATURA=int(asignatura) if asignatura else None,
                    ALUMNO=alumno if alumno else None,
                    PROFESOR=int(profesor) if profesor else None
                )
            else:
                print("Opción inválida. Intente nuevamente.")

        elif opcion == "4":
            print(
                """
                ==========================================
                |      Sistema de Gestión Académico      |
                ==========================================
                |   ¿Dónde deseas eliminar los datos?    |
                ==========================================
                | 1. Tabla Personas                      |
                | 2. Tabla Asignatura                    |
                | 3. Tabla Profesores                    |
                | 4. Tabla Cursos                        |
                | 5. Tabla Horarios                      |
                | 6. Tabla Evaluaciones                  |
                | 0. Salir                               |
                ==========================================
                """
            )
            opcion_eliminar = input("Selecciona una opcion [1-6, 0 para salir]:")
            if opcion_eliminar == "0":
                print("Regresando...")
            elif opcion_eliminar == "1":
                rut = input("Ingrese RUT de la persona a eliminar: ")
                delete_persona(rut)
            elif opcion_eliminar == "2":
                id = int(input("Ingrese ID de la asignatura a eliminar: "))
                delete_asignatura(id)
            elif opcion_eliminar == "3":
                id = int(input("Ingrese ID del profesor a eliminar: "))
                delete_profesor(id)
            elif opcion_eliminar == "4":
                id = input("Ingrese ID del curso a eliminar: ")
                delete_curso(id)
            elif opcion_eliminar == "5":
                id = int(input("Ingrese ID del horario a eliminar: "))
                delete_horario(id)
            elif opcion_eliminar == "6":
                id = int(input("Ingrese ID de la evaluación a eliminar: "))
                delete_evaluacion(id)
            else:
                print("Opción inválida. Intente nuevamente.")
if __name__ == "__main__":
    __main__()


            |      Sistema de Gestión Académico      |
            |    Bienvenido al sistema de gestión    |
            |          ¿Qué deseas hacer?:           |
            | 1. Ingresar datos                      |
            | 2. Consultar datos                     |
            | 3. Actualizar datos                    |
            | 4. Eliminar datos                      |
            | 0. Salir                               |
            

                |      Sistema de Gestión Académico      |
                |   ¿Dónde deseas ingresar los datos?    |
                | 1. Tabla Personas                      |
                | 2. Tabla Asignatura                    |
                | 3. Tabla Profesores                    |
                | 4. Tabla Cursos                        |
                | 5. Tabla Horarios                      |
                | 6. Tabla Evaluaciones                  |
                | 0. Salir                               |
              